### Counting number of the mutual friends

For each user having ID in the column userId count the amount of his / her common friends with each other user having ID in the column userId.

Print 49 pairs of the users having the largest amount of common friends, ordered in descending order first by the common friends count , then by id of user1 and finally by id of user 2.

To solve this task use the algorithm described in the last video of lesson 1.

The sample dataset is located at `/data/graphDFSample`.



In [1]:
import os
execfile(os.path.join(os.environ["SPARK_HOME"], 'python/pyspark/shell.py'))

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.1.1
      /_/

Using Python version 2.7.12 (default, Nov 19 2016 06:48:10)
SparkSession available as 'spark'.


In [2]:
from pyspark.sql import SparkSession
sparkSession = SparkSession.builder.enableHiveSupport().master("local [2]").getOrCreate()

In [3]:
graphPath = "/data/graphDFSample"

In [4]:
from pyspark.sql.functions import explode, collect_list, size, col, asc, desc, sort_array
from pyspark.sql import Window

reversedGraph = (
    sparkSession.read.parquet(graphPath)
    .withColumn("friend", explode('friends'))
    .groupBy("friend")
    .agg(collect_list("user").alias("users"))
    .withColumn("users_size", size("users"))
)

In [5]:
adjacency_lists = reversedGraph.filter(col('users_size') > 1).select(sort_array('users').alias('adj'))

In [6]:
from itertools import combinations

from pyspark.sql.functions import udf
from pyspark.sql import types


def comb(array):
    return list(combinations(array, 2))


schema = types.ArrayType(types.IntegerType())

udf_comb = udf(comb, types.ArrayType(schema))

In [7]:
combined_users = (
    adjacency_lists
        .withColumn('udf_comb', udf_comb(col('adj')))
        .select(explode('udf_comb'))
        .withColumn('a', col('col').getItem(0))
        .withColumn('b', col('col').getItem(1))
        .groupBy('a', 'b')
        .count()
        .orderBy(desc('count'), asc('a'), asc('b'))
)
# combined_users.cache()

In [ ]:
# combined_users.show(49)

In [ ]:
for row in combined_users.take(49):
    print row['a'], row['b'], row['count']

In [ ]:
# combined_users.take(1)[0]['count']